In [ ]:
# 소재지도로명주소에서 맨 앞에 위치한 도, 시 데이터를 추출하는 함수 정의
def extract_province_city(address):
    if isinstance(address, str):  # 주소가 문자열인 경우만 처리
        split_address = address.split()
        if len(split_address) >= 2:
            return split_address[0], split_address[1]  # 도, 시 반환
    return None, None  # 결측치 또는 비정상 데이터는 None 반환

# 데이터프레임에 새로운 컬럼 추가 (도, 시)
data['행정구역'], data['시'] = zip(*data['소재지도로명주소'].map(extract_province_city))

In [ ]:
region = pd.read_csv("행정구역_시군구_별_주민등록세대수_20241201130113.csv",header=0,encoding="euc-kr").drop(0)
region.columns = ["행정구역","인구수"]

In [ ]:
region

In [ ]:
data = data.merge(region,on=["행정구역"], how="inner")

In [ ]:
# 지역별 시장 개수 집계
market_count_by_region = data.groupby('행정구역')['시장명'].count().reset_index()
market_count_by_region.rename(columns={'시장명': '시장수'}, inplace=True)

# 각 도의 중심 위치(위도와 경도)를 설정하기 위한 참조 데이터 생성
region_coordinates = {
    "서울특별시": [37.5665, 126.9780],
    "부산광역시": [35.1796, 129.0756],
    "대구광역시": [35.8714, 128.6014],
    "인천광역시": [37.4563, 126.7052],
    "광주광역시": [35.1595, 126.8526],
    "대전광역시": [36.3504, 127.3845],
    "울산광역시": [35.5384, 129.3114],
    "세종특별자치시": [36.4800, 127.2890],
    "경기도": [37.4138, 127.5183],
    "강원도": [37.8228, 128.1555],
    "충청북도": [36.6353, 127.4914],
    "충청남도": [36.5184, 126.8000],
    "전라북도": [35.7175, 127.1530],
    "전남특별자치도": [34.8679, 126.9910],
    "경상북도": [36.4919, 128.8889],
    "경상남도": [35.4606, 128.2132],
    "제주특별자치도": [33.4996, 126.5312]
}

# 도별 시장수 데이터에 위도와 경도를 추가
market_count_by_region['위도'] = market_count_by_region['행정구역'].map(lambda x: region_coordinates.get(x, [0, 0])[0])
market_count_by_region['경도'] = market_count_by_region['행정구역'].map(lambda x: region_coordinates.get(x, [0, 0])[1])

# 2023년 인구수 데이터
# 각 도의 인구수 데이터 (참조 데이터)
population_data = {
    "서울특별시": 9384512,
    "부산광역시": 3279604,
    "대구광역시": 2379188,
    "인천광역시": 3025950,
    "광주광역시": 1457090,
    "대전광역시": 1470336,
    "울산광역시": 1107432,
    "세종특별자치시": 386261,
    "경기도": 13815367,
    "강원도": 1528014,
    "충청북도": 1641481,
    "충청남도": 2216332,
    "전라북도": 1768491,
    "전남특별자치도": 1776668,
    "경상북도": 2589880,
    "경상남도": 3271148,
    "제주특별자치도": 676767
}

# 인구수 데이터를 추가
market_count_by_region['인구수'] = market_count_by_region['행정구역'].map(population_data)

# 인구 10만 명당 시장 수 계산
market_count_by_region['인구 10만 명당 시장수'] = (market_count_by_region['시장수'] / market_count_by_region['인구수']) * 100000

# 지도 시각화 (인구 대비 시장수 기준)
korea_map = folium.Map(location=[36.5, 127.5], zoom_start=7)


# 지역별 시장 밀도를 원의 크기로 표시
for idx, row in market_count_by_region.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=row['인구 10만 명당 시장수'] * 8,  # 비율을 크기에 반영 (스케일링)
        fill=True,
        fill_opacity=0.6,
        popup=(
            f"{row['행정구역']}<br>"
            f"시장수: {row['시장수']}<br>"
            f"인구수: {row['인구수']:,}<br>"
            f"인구 10만 명당 시장수: {row['인구 10만 명당 시장수']:.2f}"
        ),
    ).add_to(korea_map)

# 지도 출력
korea_map

In [ ]:
import pandas as pd

# Given data
data = {
    "Year": [2021, 2022],
    "Population": [51738071, 51692272],
    "Average_Daily_Customers": [4672, 4536],
    "Market_Daily_Sales": [57460000, 57700000],  # Converted to KRW
    "Food_Beverage_Sales": [24554892000, 26334349000]  # Already in KRW
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Calculate annualized sales (assuming 5 days per week and 52 weeks per year)
df["Market_Annual_Sales"] = df["Market_Daily_Sales"] * 5 * 52
df["Food_Beverage_Annual_Sales"] = df["Food_Beverage_Sales"]


import matplotlib.pyplot as plt

# Data for the chart
years = df["Year"]
population = df["Population"]
avg_customers = df["Average_Daily_Customers"]
market_sales = df["Market_Annual_Sales"]
food_beverage_sales = df["Food_Beverage_Annual_Sales"]

In [ ]:
# Create separate bar charts for each metric
metrics = {
    "Population": population,
    "Avg Daily Customers": avg_customers,
    "Market Annual Sales": market_sales,
    "Food/Beverage Sales": food_beverage_sales,
}

# Correcting the logic to ensure the comparison is properly structured with one metric and two bars for 2021 vs 2022
for metric, values in metrics.items():
    plt.figure(figsize=(8, 5))

    x = ["2021", "2022"]  # Years as categories
    bar_width = 0.4  # Slim bar width

    # Plotting bars for 2021 and 2022
    plt.bar(x, values, width=bar_width, color=["blue", "orange"], alpha=0.7, label=["2021", "2022"])

    # Adding line for comparison
    plt.plot(x, values, color="black", linestyle="--", marker="o", label="Trend Line")

    # Title and labels
    plt.title(f"{metric}: 2021 vs 2022", fontsize=14, fontweight="bold")
    plt.ylabel("Values (KRW or Count)", fontsize=12)
    plt.xlabel("Year", fontsize=12)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=10)

    plt.tight_layout()
    plt.show()